In [3]:
import random

In [4]:
class BoardException(Exception):
    pass


class BoardOutException(BoardException):
    pass


class BoardUsedException(BoardException):
    pass


class BoardWrongShipException(BoardException):
    pass

In [5]:
class Dot:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return f"Dot({self.x}, {self.y})"


In [6]:
class Ship:
    def __init__(self, bow, length, orientation):
        self.bow = bow
        self.length = length
        self.orientation = orientation
        self.lives = length

    @property
    def dots(self):
        ship_dots = []
        for i in range(self.length):
            cur_x = self.bow.x
            cur_y = self.bow.y

            if self.orientation == 0:
                cur_x += i

            elif self.orientation == 1:
                cur_y += i

            ship_dots.append(Dot(cur_x, cur_y))

        return ship_dots


In [7]:
class Board:
    def __init__(self, hid=False, size=6):
        self.size = size
        self.hid = hid

        self.count = 0

        self.field = [["O"] * size for _ in range(size)]

        self.busy = []
        self.ships = []

    def add_ship(self, ship):

        for d in ship.dots:
            if self.out(d) or d in self.busy:
                raise BoardWrongShipException()
        for d in ship.dots:
            self.field[d.x][d.y] = "■"
            self.busy.append(d)

        self.ships.append(ship)
        self.contour(ship)

    def contour(self, ship, verb=False):
        near = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1), (0, 0), (0, 1),
            (1, -1), (1, 0), (1, 1)
        ]
        for d in ship.dots:
            for dx, dy in near:
                cur = Dot(d.x + dx, d.y + dy)
                if not (self.out(cur)) and cur not in self.busy:
                    if verb:
                        self.field[cur.x][cur.y] = "T"
                    self.busy.append(cur)

    def __str__(self):
        res = ""
        res += "  | 1 | 2 | 3 | 4 | 5 | 6 |"
        for i, row in enumerate(self.field):
            res += f"\n{i + 1} | " + " | ".join(row) + " |"

        if self.hid:
            res = res.replace("■", "O")
        return res

    def out(self, d):
        return not ((0 <= d.x < self.size) and (0 <= d.y < self.size))

    def shot(self, d):
        if self.out(d):
            raise BoardOutException()

        if d in self.busy:
            print('''Вы уже стреляли в данную клетку. 
       Выберите другую''')
            raise BoardUsedException()

        self.busy.append(d)

        for ship in self.ships:
            if d in ship.dots:
                ship.lives -= 1
                self.field[d.x][d.y] = "X"
                if ship.lives == 0:
                    self.count += 1
                    self.contour(ship, verb=True)
                    print("Корабль уничтожен!")
                    return False
                else:
                    print("Корабль ранен!")
                    return True

        self.field[d.x][d.y] = "T"
        print("Мимо!")
        return False

    def begin(self):
        self.busy = []


In [8]:
class Player:
    def __init__(self, board, enemy):
        self.board = board
        self.enemy = enemy

    def ask(self):
        raise NotImplementedError()

    def move(self):
        while True:
            try:
                target = self.ask()
                repeat = self.enemy.shot(target)
                return repeat
            except BoardException as e:
                print(e)

In [9]:
class AI(Player):
    def ask(self):
        d = Dot(random.randint(0, 5), random.randint(0, 5))
        print(f"Ход компьютера: {d.x + 1} {d.y + 1}")
        return d

In [10]:
class User(Player):
    def ask(self):
        while True:
            cords = input("Ваш ход: ").split()

            if len(cords) != 2:
                print(" Введите 2 координаты! ")
                continue

            x, y = cords

            if not (x.isdigit()) or not (y.isdigit()):
                print(" Введите числа! ")
                continue

            x, y = int(x), int(y)

            if x < 1 or x > 6 or y < 1 or y > 6:
                print(" Введите координаты в диапазоне от 1 до 6! ")
                continue

            return Dot(x - 1, y - 1)


In [11]:
class Game:
    def __init__(self, size=6):
        self.size = size
        pl = self.random_board()
        co = self.random_board()
        co.hid = True

        self.ai = AI(co, pl)
        self.us = User(pl, co)

    def random_board(self):
        board = None
        while board is None:
            board = self.try_board()
        return board

    def try_board(self):
        len1 = [3]
        len2 = [2, 2]
        len3 = [1, 1, 1, 1]
        lens = len1 + len2 + len3
        board = Board(size=self.size)
        attempts = 0
        for l in lens:
            while True:
                attempts += 1
                if attempts > 2000:
                    return None
                ship = Ship(Dot(random.randint(0, self.size), random.randint(0, self.size)), l, random.randint(0, 1))
                try:
                    board.add_ship(ship)
                    break
                except BoardWrongShipException:
                    pass
        board.begin()
        return board

    def greet(self):
        print("---------------------------")
        print("  Добро пожаловать в игру  ")
        print("       'Морской бой'   ")
        print("---------------------------")
        print(" формат ввода: x y ")
        print(" x - номер строки  ")
        print(" y - номер столбца ")

    def loop(self):
        num = 0
        while True:
            print("-" * 27)
            print("Доска пользователя:")
            print(self.us.board)
            print("-" * 27)
            print("Доска компьютера:")
            print(self.ai.board)
            if num % 2 == 0:
                print("-" * 27)
                print("Ходит пользователь!")
                repeat = self.us.move()
            else:
                print("-" * 27)
                print("Ходит компьютер!")
                repeat = self.ai.move()
            if repeat:
                num -= 1
            if self.ai.board.count == 7:
                print("-" * 27)
                print("Пользователь выиграл!")
                break
            if self.us.board.count == 7:
                print("-" * 27)
                print("Компьютер выиграл!")
                break
            num += 1

    def start(self):
        self.greet()
        self.loop()


In [ ]:
g = Game()
g.start()

---------------------------
  Добро пожаловать в игру  
       'Морской бой'   
---------------------------
 формат ввода: x y 
 x - номер строки  
 y - номер столбца 
---------------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | ■ | O | O | ■ | O | ■ |
2 | O | O | O | ■ | O | O |
3 | ■ | ■ | O | O | O | ■ |
4 | O | O | O | O | O | O |
5 | O | O | O | ■ | ■ | ■ |
6 | ■ | O | O | O | O | O |
---------------------------
Доска компьютера:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | O | O | O | O | O | O |
2 | O | O | O | O | O | O |
3 | O | O | O | O | O | O |
4 | O | O | O | O | O | O |
5 | O | O | O | O | O | O |
6 | O | O | O | O | O | O |
---------------------------
Ходит пользователь!
Ваш ход: 2 2
Мимо!
---------------------------
Доска пользователя:
  | 1 | 2 | 3 | 4 | 5 | 6 |
1 | ■ | O | O | ■ | O | ■ |
2 | O | O | O | ■ | O | O |
3 | ■ | ■ | O | O | O | ■ |
4 | O | O | O | O | O | O |
5 | O | O | O | ■ | ■ | ■ |
6 | ■ | O | O | O | O | O |
---------------------------
Доска к